In [1]:
import torch
from bfgs_utils import *
# from utils import *
from scipy.optimize import minimize, least_squares, NonlinearConstraint
printify = lambda f: lambda *args, **kwargs: print(f(*args, **kwargs), end='\n' + '-'*50 + '\n')

In [2]:
pminimize = printify(minimize)

pminimize(lambda x: (x[0] - torch.pi)**2 + (x[1] + torch.pi/2)**2, [0., 0.])
pminimize(lambda x: ((x[0]-2) + (x[1]-3))**2, [0., 0.])
pminimize(lambda x: ((x[0]-2) + (x[1]-3))**2, [-2., 0.], bounds=[(-np.inf,-4), (-np.inf,np.inf)])
pminimize(lambda x: ((x[0]-2) + (x[1]-3))**2, [-5., 0.], bounds=[(-np.inf,-4), (-np.inf,np.inf)])
pminimize(lambda x: ((x[0]-2) + (x[1]-3))**2, [-5., 0.], bounds=[(-4,-4), (-np.inf,np.inf)])

      fun: 2.7605759509528782e-14
 hess_inv: array([[0.60000004, 0.20000003],
       [0.20000003, 0.89999997]])
      jac: array([1.57835371e-07, 3.14889244e-07])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 3
     njev: 4
   status: 0
  success: True
        x: array([ 3.14159273, -1.57079618])
--------------------------------------------------
      fun: 3.542141690395744e-17
 hess_inv: array([[ 0.625, -0.375],
       [-0.375,  0.625]])
      jac: array([2.99798231e-09, 2.99798231e-09])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 3
     njev: 4
   status: 0
  success: True
        x: array([2.5, 2.5])
--------------------------------------------------
      fun: 2.500003966417107e-17
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.00000079e-08, -7.10542736e-15])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 12
      nit: 3
     njev: 4
   status: 0
  success: True
  

In [3]:
pleast_squares = printify(least_squares)

pleast_squares(lambda x: (x[0] - torch.pi)**2 + (x[1] + torch.pi/2)**2, [0., 0.])
pleast_squares(lambda x: ((x[0]-2) + (x[1]-3))**2, [0., 0.])
pleast_squares(lambda x: ((x[0]-2) + (x[1]-3))**2, [-4., 0.], bounds= ([-np.inf, -np.inf], [-4, np.inf]))
pleast_squares(lambda x: ((x[0]-2) + (x[1]-3))**2, [-4., 0.], bounds= ([-4 - 1e-10, -np.inf], [-4., np.inf]))


 active_mask: array([0., 0.])
        cost: 1.2783752744478993e-12
         fun: array([1.59898422e-06])
        grad: array([ 3.61701446e-09, -1.80850731e-09])
         jac: array([[ 0.00226207, -0.00113104]])
     message: '`gtol` termination condition is satisfied.'
        nfev: 17
        njev: 10
  optimality: 3.6170144554807505e-09
      status: 1
     success: True
           x: array([ 3.14272367, -1.57136183])
--------------------------------------------------
 active_mask: array([0., 0.])
        cost: 2.5731570692902734e-13
         fun: array([7.17378153e-07])
        grad: array([1.21523919e-09, 1.21523919e-09])
         jac: array([[0.001694, 0.001694]])
     message: '`gtol` termination condition is satisfied.'
        nfev: 16
        njev: 11
  optimality: 1.2152391925996551e-09
      status: 1
     success: True
           x: array([2.50042349, 2.50042349])
--------------------------------------------------
 active_mask: array([0, 0])
        cost: 3.891221023383139e

In [4]:
coordinates = (torch.tensor([-2, -1, 0, 1, 2, 3]),)
values = torch.tensor([16, 1, 0, 1, 16, 81], )
print(torch.gradient(values, spacing = coordinates))
print(torch.gradient(values, spacing = coordinates, edge_order=2))


(tensor([-15.,  -8.,   0.,   8.,  40.,  65.]),)
(tensor([-22.,  -8.,   0.,   8.,  40.,  90.]),)


In [5]:
def calc_poly(coeffs, points):
    return sum(coeffs[i]*points**i for i in range(len(coeffs)))

def test_torch_pdl(coeffs, points, **config):
    coeffs, points = map(torch.tensor, [coeffs, points])
    applied = calc_poly(coeffs, points)
    f = lambda x: torch.linalg.norm(applied - calc_poly(x, points))
    pleast_squares(f, [0]*len(coeffs), method='dogbox', **config)

test_torch_pdl([1, 0, 1], [1, 0, -1])
test_pdl([1, 0, 1], [1, 0, -1], epoch=40, tr0=1)
print('-'*50)

 active_mask: array([0, 0, 0])
        cost: 5.467451202889606e-08
         fun: array([0.00033068], dtype=float32)
        grad: array([ 0.0001163 , -0.00025003, -0.00014195])
         jac: array([[ 0.35171437, -0.75611669, -0.42926136]])
     message: '`xtol` termination condition is satisfied.'
        nfev: 48
        njev: 42
  optimality: 0.00025003239616096587
      status: 3
     success: True
           x: array([ 1.00007520e+00, -1.72101474e-06,  9.99697044e-01])
--------------------------------------------------
PDL result for [1 0 1] is
[0.99999952 0.         0.9999995 ]
--------------------------------------------------


In [21]:
wrap = lambda f: lambda args: f(*args)

from scipy.optimize import rosen
x0 = [1.3, 0.7, 0.8, 1.9, 1.2]
pminimize(rosen, x0, method='BFGS', tol=1e-6)

f = lambda size : lambda x,y : (x + 2*y - 7)**2 + (2*x + y - 5)**2
grad = lambda x, y: [2*(x+2*y - 7)+2*2*(2*x + y -5), 2*2*(x+2*y-7) + 2*(2*x + y - 5)]
print(BFGS(f, 0, [5,5], grad, 10)[-1])
print('-'*50)

pminimize(wrap(f(0)), [5,5], method='BFGS', tol=1e-6)







      fun: 4.5818709873963196e-11
 hess_inv: array([[0.00763893, 0.01252135, 0.0236093 , 0.04659247, 0.0932239 ],
       [0.01252135, 0.02490652, 0.04729656, 0.09358024, 0.18715678],
       [0.0236093 , 0.04729656, 0.09486791, 0.18792233, 0.37576475],
       [0.04659247, 0.09358024, 0.18792233, 0.377341  , 0.75448228],
       [0.0932239 , 0.18715678, 0.37576475, 0.75448228, 1.51357483]])
      jac: array([-5.78899122e-06, -2.82251170e-06, -2.79807396e-06, -7.64314158e-06,
        5.85476809e-06])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 372
      nit: 25
     njev: 60
   status: 2
  success: False
        x: array([0.99999925, 0.99999852, 0.99999706, 0.99999416, 0.99998833])
--------------------------------------------------
1
2
3
4
5
6
7
[0.99999992 3.00000002]
--------------------------------------------------
      fun: 3.510679474496521e-16
 hess_inv: array([[ 0.27792958, -0.22208688],
       [-0.22208688,  0.27789844]])
      jac: array